In [13]:
from ethnicitydetector import EthnicityDetector
import pandas as pd
from multiprocessing import Pool
import numpy as np
import time
from collections import Counter

In [14]:
ed = EthnicityDetector()

In [15]:
for n in ed.ETHNICITY_LIST:
    print(n)

indian
japanese
greek
arabic
turkish
thai
vietnamese
balkan
italian
samoan
hawaiian
khmer
chinese
korean


In [16]:
d = pd.read_csv("/Users/ik/Data/customers/all-customers-18092017.csv.gz", encoding='latin-1',dtype=str, error_bad_lines=False)
d["full_name"] = d["first_name"].fillna('') + ' ' + d["middle_name"].fillna('') + ' ' + d["last_name"].fillna('')

In [17]:
b = d

# t0 = time.time()
# b["ethnicity"] = b["full_name"].apply(ed.get_ethnicity)
# print("elapsed time: {:.0f} min {:.0f} sec".format(*divmod(time.time() - t0, 60)))

vf = np.vectorize(ed.get_ethnicity)

In [18]:
def get_array_ethnicity(b):  
    """
    IN: numpy array that has two columns, oned contains customer id and another a full name
    OUT: numpy array with teo columns: customer id and ethnicity
    
    !NOTE: there will be 'None' where no ethnicityhas been detected 
    """

    ets = vf(b[:,-1])  # we assume that the second column contains full names
    stk = np.hstack((b[:,0].reshape(b.shape[0],1), ets.reshape(b.shape[0],1)))

    return stk

t0 = time.time()

pool = Pool(2)
res = pd.DataFrame(np.vstack(pool.map(get_array_ethnicity, np.array_split(b.loc[:, ["cust_id","full_name"]].values, 2))),
                   columns=["CustomerID", "Ethnicity"], dtype=str).query('Ethnicity != "None"')
pool.close()
pool.join()

print("elapsed time: {:.0f} min {:.0f} sec".format(*divmod(time.time() - t0, 60)))

elapsed time: 6 min 52 sec


In [21]:
res.head()

,CustomerID,Ethnicity
0,3161590,italian
1,3161606,
2,3161623,
3,3161636,
4,3161641,


In [22]:
Counter(res.Ethnicity).most_common(30)

[('', 17180779),
 ('korean', 334702),
 ('chinese', 295149),
 ('italian', 292394),
 ('indian', 209432),
 ('chinese|korean', 178657),
 ('arabic', 107944),
 ('greek', 94229),
 ('japanese', 88886),
 ('balkan', 84376),
 ('vietnamese', 80968),
 ('turkish', 42159),
 ('arabic|indian', 27677),
 ('arabic|indian|turkish', 21392),
 ('arabic|turkish', 14142),
 ('chinese|korean|vietnamese', 9323),
 ('hawaiian', 8207),
 ('thai', 6047),
 ('korean|vietnamese', 5578),
 ('khmer', 5509),
 ('samoan', 4891),
 ('japanese|korean', 4578),
 ('indian|italian', 2889),
 ('chinese|vietnamese', 2877),
 ('chinese|thai', 2743),
 ('chinese|japanese', 2211),
 ('indian|japanese', 1789),
 ('chinese|japanese|korean', 1724),
 ('arabic|italian', 1715),
 ('arabic|japanese', 1699)]

In [8]:
t0 = time.time()

b["Ethnicity"] = b["full_name"].apply(ed.get_ethnicity)

print("elapsed time: {:.0f} min {:.0f} sec".format(*divmod(time.time() - t0, 60)))

elapsed time: 9 min 48 sec


In [34]:
Counter(res1.Ethnicity).most_common(15)

[('korean', 334702),
 ('chinese', 295149),
 ('italian', 292394),
 ('indian', 209432),
 ('chinese|korean', 178657),
 ('arabic', 107944),
 ('greek', 94229),
 ('japanese', 88886),
 ('balkan', 84376),
 ('vietnamese', 80968),
 ('turkish', 42159),
 ('arabic|indian', 27677),
 ('arabic|indian|turkish', 21392),
 ('arabic|turkish', 14142),
 ('chinese|korean|vietnamese', 9323)]

In [24]:
import arrow
TODAY_SYD = arrow.utcnow().to('Australia/Sydney').format('DD-MM-YYYY')

In [26]:
res['AssignedOn'] = TODAY_SYD

In [30]:
res1 = res.loc[res.Ethnicity.apply(lambda x: len(x) > 3)]

In [31]:
len(res1)

1961415

In [32]:
len(res)

19142194

In [33]:
res1.to_csv('CustomerEthnicities.csv', index=False)

In [29]:
d1.loc[:,["cust_id", "full_name"]].sample(n=200000).to_csv("sample_other.csv", index=None)

In [44]:
nocn = pd.concat([pd.read_csv("sample_arabic.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_balkan.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_greek.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_hawaiian.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_indian.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_italian.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_japanese.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_khmer.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_samoan.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_turkish.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_vietnamese.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner"),
                 pd.read_csv("sample_korean.csv", dtype=str)
                  .join(d.loc[:,["cust_id", "full_name"]]
                        .set_index("cust_id"), on="CustomerID", how="inner")])

In [55]:
oz = oz.rename(columns={"full_name": "FullName", "cust_id": "CustomerID"})

In [56]:
oz.head()

,CustomerID,FullName
0,22270069,rachel hill
1,24101733,erica lawrence
2,16313263,louise otoole
3,25558717,anthony roberts
4,21833584,angela huggins


In [60]:
w.to_csv("ethnicities_not_chinese.csv", index=False)

In [49]:
oz = pd.read_csv("sample_other.csv", dtype=str)

In [50]:
oz.head()

,cust_id,full_name
0,22270069,rachel hill
1,24101733,erica lawrence
2,16313263,louise otoole
3,25558717,anthony roberts
4,21833584,angela huggins


In [57]:
w = pd.concat([nonc.loc[:,["CustomerID", "FullName"]], oz])

In [58]:
w.head()

,CustomerID,FullName
0,18467850,hadi sedaqat
1,25251693,taj jones
2,23830043,barak kessar
3,25327007,natalie mazin
4,17472846,salma begg


In [59]:
len(w)

568000

In [62]:
cn_1 = pd.read_csv("CBDB_chinese_names.txt", header=None, encoding='latin-1')

In [64]:
cn_1["Ethnicity"] = "chinese"

In [65]:
cn_1 = cn_1.rename(columns={0:"FullName"})

In [68]:
cn_1.head()

,FullName,Ethnicity
0,an dun,chinese
1,an fang,chinese
2,an tao,chinese
3,zha dao,chinese
4,zha yue,chinese


In [67]:
cn_1 = cn_1[["FullName", "Ethnicity"]]

In [69]:
cn_2 = pd.read_csv("chinese2.csv", header=None, encoding='latin-1')

In [70]:
cn_2 = cn_2.rename(columns={0:"FullName"})

In [73]:
cn_2.head()

,FullName,Ethnicity
0,yuxiao zhang,chinese
1,robin chen,chinese
2,ching khow,chinese
3,kuo yuchen,chinese
4,wei-chi yeh,chinese


In [72]:
cn_2["Ethnicity"] = "chinese"

In [74]:
cn = pd.concat([cn_1,cn_2])

In [75]:
cn.head()

,FullName,Ethnicity
0,an dun,chinese
1,an fang,chinese
2,an tao,chinese
3,zha dao,chinese
4,zha yue,chinese


In [76]:
cn.drop_duplicates("FullName", inplace=True)

In [77]:
len(cn)

240396

In [100]:
w.head()

,CustomerID,FullName
0,18467850,hadi sedaqat
1,25251693,taj jones
2,23830043,barak kessar
3,25327007,natalie mazin
4,17472846,salma begg


In [79]:
w["FullName"].to_csv("training_nocn.csv", index=False)

In [80]:
cn["FullName"].to_csv("training_iscn.csv", index=False)

In [81]:
cn.head()

,FullName,Ethnicity
0,an dun,chinese
1,an fang,chinese
2,an tao,chinese
3,zha dao,chinese
4,zha yue,chinese


In [82]:
w.head()

,CustomerID,FullName
0,18467850,hadi sedaqat
1,25251693,taj jones
2,23830043,barak kessar
3,25327007,natalie mazin
4,17472846,salma begg


In [90]:
training_cn["FullName"] = cn["FullName"]
training_cn["isChinese"] = 1

In [88]:
training_not = w["FullName"]
training_not.loc[:, "isChinese"] = 0

IndexingError: (slice(None, None, None), 'isChinese')

In [85]:
training = pd.concat([training_cn, training_not])

In [86]:
training.head()

0     an dun
1    an fang
2     an tao
3    zha dao
4    zha yue
Name: FullName, dtype: object

In [91]:
training_cn.head()

0     an dun
1    an fang
2     an tao
3    zha dao
4    zha yue
Name: FullName, dtype: object

In [89]:
w["FullName"]

0                    hadi  sedaqat
1                       taj  jones
2                    barak  kessar
3                   natalie  mazin
4                      salma  begg
5                  andrew  barakat
6                   habib  iddrisu
7                  kamil  brzostek
8                   dana  lepinzan
9                  nevine  khalifa
10                   khoder  samia
11              jean-pierre  mahÃ©
12                    yazid  habib
13                      dana  meos
14                 bradley  haider
15                 nurziana  jamil
16                 amani  zeitoune
17                       ayu  hadi
18             nadia   fitzpatrick
19                    nadia   gray
20                   nurul  roslan
21                   abdul  hassan
22                   osman  hassan
23                 hashrul  rashid
24                    fatima  safa
25                   iqbal  yaseer
26                   justin  hanan
27                  dana  fluitsma
28                  

In [92]:
nocn = pd.read_csv("training_nocn.csv")

In [93]:
nonc

,CustomerID,Ethnicity,FullName
0,18467850,arabic,hadi sedaqat
1,25251693,arabic,taj jones
2,23830043,arabic,barak kessar
3,25327007,arabic,natalie mazin
4,17472846,arabic,salma begg
5,18262666,arabic,andrew barakat
6,19897797,arabic,habib iddrisu
7,16605979,arabic,kamil brzostek
8,20142429,arabic,dana lepinzan
9,25702683,arabic,nevine khalifa


In [96]:
ch = pd.read_csv("training_iscn.csv", header=None)

In [98]:
noch = pd.read_csv("training_nocn.csv", header=None)

In [101]:
ch["is_chinese"] = 1
noch["is_chinese"] = 0

In [102]:
ch.head()

,0,is_chinese
0,an dun,1
1,an fang,1
2,an tao,1
3,zha dao,1
4,zha yue,1


In [103]:
data = pd.concat([ch, noch])

In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808391 entries, 0 to 567994
Data columns (total 2 columns):
0             808391 non-null object
is_chinese    808391 non-null int64
dtypes: int64(1), object(1)
memory usage: 18.5+ MB


In [106]:
data = data.rename(columns={0: "full_name"})

In [107]:
data.head()

,full_name,is_chinese
0,an dun,1
1,an fang,1
2,an tao,1
3,zha dao,1
4,zha yue,1


In [108]:
Counter(data.is_chinese)

Counter({0: 567995, 1: 240396})

In [109]:
data.to_csv("tranining_chinese.csv", index=False)